In [2]:
def ReadIn (name):
    table = pd.read_csv(name + '.csv', header = None)
    time = pd.read_table(name + '.z.txt', header = None)
    wavelength = pd.read_table(name + '.x.txt', header = None)
    return (table, time, wavelength)

In [19]:
def Dictionary_of_Prepped_Spectra(identifier, allSpectraDict, allDataDict, TRP, paramsDict):
    Prepped_Spectra = {}
    timeAndWLDict ={}
    sumCenters = paramsDict['beg']['center']+paramsDict['end']['center']
    sumSigmas = paramsDict['beg']['sigma']+paramsDict['end']['sigma']

    peak_beg = sumCenters/2-(sumSigmas/2)*2.3548200
    peak_end = sumCenters/2+(sumSigmas/2)*2.3548200
    spectraDict = allSpectraDict[identifier]
    TRP[identifier] = {'peak_beg':peak_beg,'peak_end':peak_end}
    for key in spectraDict.keys():
        spectrum = spectraDict[key]
        time = allDataDict[identifier][key]['time']
        wl = allDataDict[identifier][key]['wl']
        Out_spectrum, time, wl = PrepareSpectra(spectrum, time, wl, 
                                                    TRP['b'],TRP['b2'], 
                                                    TRP['left_bound'], TRP['right_bound'], 
                                                    TRP['ave_n'], TRP['poly_n'], 
                                                    peak_beg, peak_end)

        Prepped_Spectra[key] = Out_spectrum
        timeAndWLDict = {'time':time, 'wl':wl}
    return Prepped_Spectra, timeAndWLDict, TRP

In [5]:
"""
Use this function to take a dictionary of prepped spectra (already background and polynomial corrected) and get an average spectrum
"""
def average_of_prepped_spectra(dictionary, identifier):
    spectra_keys = list(dictionary[identifier].keys())
    Sum = dictionary[identifier][spectra_keys[0]]-dictionary[identifier][spectra_keys[0]]
    for key in spectra_keys:
        Sum += dictionary[identifier][key]
    table_mean = Sum/len(spectra_keys)
    return table_mean

In [11]:
"""
This function takes a Time Resolved spectrum given as a dataframe (pd.read_csv), averages over a rolling average specified by n, then subtracts the mean of the "before injection" period, fits and subtracts a polynomial and outputs the converted spectra and relevant time file. Reasonable way to call this function is as follows: PrepareSpectra(test, wllDF, 21,21, 104, 256,100, 4)
"""
def PrepareSpectra (TRspec, time, wl, before, beginning, endpoint_wl2, endpoint_wl1,ave_n, poly_n, wlLow, wlHigh):
    beg = beginning
    end1 = WLtoIndex(wl, endpoint_wl1)
    end2 = WLtoIndex(wl, endpoint_wl2)
    S = averaging(TRspec.iloc[beg:, end1:end2], ave_n) - np.mean(TRspec.iloc[0:before, end1:end2].values, axis = 0)
    new_table = polyfit('TR', S, wl.iloc[end1:end2,:], poly_n, wlLow, wlHigh)
    
    AdjTime = averaging(time.iloc[0:len(time)-beg], ave_n)
    TimeArray = AdjTime.values.T[0]
    return (new_table, TimeArray, wl.iloc[end1:end2,:])

In [12]:
"""
This function takes a Time Resolved spectrum given as a dataframe (pd.read_csv) and averages over a rolling average specified by ave_n.
"""
def averaging(df,ave_n, **kwargs):  
    new_df = pd.DataFrame()
    reps = len(df)
    start = ave_n
    finish = len(df)+1
    for j in np.arange(start, finish):
        sector = df[(j-(ave_n)):j]
        aves = sector.mean(axis = 0)
        aves_T = pd.DataFrame(aves).T
        new_df = new_df.append(aves_T)
        
    return new_df

In [8]:
def PeakFromSingleGaussian(identifier, All_Prepped_Spectra, tAndWL, paramsDict):
    def _1gaussian(x, amp1,cen1,sigma1):
        return amp1*(1/(sigma1*(np.sqrt(2*np.pi))))*(np.exp((-1.0/2.0)*(((x-cen1)/sigma1)**2)))
    time, wl = getTimeAndWL(identifier, tAndWL)
    peaks = []
    ts = []
    begParams = paramsDict[identifier]['beg']
    endParams = paramsDict[identifier]['end']
    begAmp = begParams['amplitude']
    endAmp = endParams['amplitude']
    begSigma = begParams['sigma']
    endSigma = endParams['sigma']
    begCenter = begParams['center']
    endCenter = endParams['center']

    amp = np.mean([begAmp, endAmp])
    sigma = np.mean([begSigma, endSigma])
    center = np.mean([begCenter, endCenter])

    spectrum = average_of_prepped_spectra(All_Prepped_Spectra, identifier)

    for t in np.arange(0,np.shape(spectrum)[0],1):
        y = np.zeros(len(spectrum.iloc[0]))
        x = np.zeros(len(wl))
        for i in range(len(spectrum.iloc[0])):
            y[i] = spectrum.iloc[t, i]
            x[i] = wl.iloc[i]
        data = np.column_stack([x,y])

        dat = data
        x = dat[:, 0]
        y = dat[:, 1]

        popt_gauss, pcov_gauss = scipy.optimize.curve_fit(_1gaussian, x, y, p0=[amp, center, sigma])

        perr_gauss = np.sqrt(np.diag(pcov_gauss))
        peaks = np.append(peaks, popt_gauss[1])
        ts = np.append(ts, time[t])
    return(peaks, ts)

In [20]:
def PeakFromHalfIntegral(identifier, All_Prepped_Spectra, tAndWL, TRP):
    spectrum = average_of_prepped_spectra(All_Prepped_Spectra, identifier)
    time, wl = getTimeAndWL(identifier, tAndWL)
    wlLow = TRP[identifier]['peak_beg']
    wlHigh = TRP[identifier]['peak_end']
    k = WLtoIndex(wl, wlLow)
    j = WLtoIndex(wl, wlHigh)
    wl_testSize = wl.iloc[j:k]
    
    peaks = []
    ts = []
    for t in np.arange(0,np.shape(spectrum)[0],1):

        time_slice = spectrum.iloc[t, j:k]
        total = np.trapz(time_slice)
        percents = []
        for i in range(len(time_slice)):
            integral = np.trapz(time_slice[0:i])/total
            percents=np.append(percents, integral)
        def test_func(w, m, b,c):
            return m/(b+(np.exp(-c*w)))
        params, params_covariance = scipy.optimize.curve_fit(test_func, np.arange(k-j), percents, p0=[-1, 0.5, 2])
        peakpos = (-1/params[2])*np.log((params[0]-0.5*params[1])/0.5) 
        peak = wlHigh - peakpos*(wlHigh-wlLow)/(k-j)
        peaks = np.append(peaks, peak)
        ts = np.append(ts, time[t])
    return peaks, ts

In [21]:
def findPeakWithSingleGaussianFit(All_Prepped_Spectra, tAndWL, identifier, description, paramsDict, pdf):
    peaks, time = PeakFromSingleGaussian(identifier, All_Prepped_Spectra, tAndWL, paramsDict)
    plot_any_peaks_over_time(time, peaks, description, with_plot = True, with_report = True, to_save = True)
    plt.title('$\\tau_g$ = 9.6 $\pm$ 3.4 min  ' , loc = 'Right', y = 0.85, fontsize = 14)
    pdf.savefig()



def findPeaksAtPointofHalfTotalIntegration(identifier, All_Prepped_Spectra, tAndWL, TRP, description, pdf):
    peaks, time = PeakFromHalfIntegral(identifier, All_Prepped_Spectra, tAndWL, TRP)
    plot_any_peaks_over_time(time, peaks, description, pdf, to_save = False,  with_plot = True, with_report = True)


In [ ]:
#Functions for preparing raw time-resolved data for plotting

def getTRPVariables(allRawData):
    print("Beginning indicates the scan number at which the “background” preinjection section stops. Background 2 indicates which scan number will be the first data point included in analysis.")
    b = eval(input('Enter beginning: '))
    b2 = eval(input('Enter beginning 2: '))
    print("Please give bounds for the region of interest")
    left_bound = eval(input('Enter lower wavenumber bound: '))
    right_bound = eval(input('Enter upper wavenumber bound: '))
    print("The average number determines the number of spectra included in a rolling average to improve signal to noise in the data.")
    ave_n = eval(input('Enter average number: '))
    print("The poly number determines the order of the polynomial subtracted from the data to correct the baseline.")
    poly_n = eval(input('Enter poly number: '))
    TRP = {'b':b,'b2':b2,'left_bound':left_bound,'right_bound':right_bound,'ave_n':ave_n,'poly_n':poly_n}
    return TRP


"""
Returns a dictionary of prepped spectra and a separate dictionary of respective times and wavelengths.
"""
def prepareAllTimeResolvedRawData(allRawData, allRawSpectraDict, paramsDict, TRP):
    All_Prepped_Spectra = {}
    AllTimeAndWL = {}
    for key in allRawSpectraDict.keys():
        params = paramsDict[str(key)]
        preppedSpectra, timeAndWL, TRP = Dictionary_of_Prepped_Spectra(key, allRawSpectraDict, allRawData, TRP, params)
        All_Prepped_Spectra[key] = preppedSpectra
        AllTimeAndWL[key] = timeAndWL
    return All_Prepped_Spectra, AllTimeAndWL, TRP


"""
Retrieves the time and wavelength for a specific identifier.
"""
def getTimeAndWL(identifier, AllTimeAndWL):
    time = AllTimeAndWL[identifier]['time']
    wl = AllTimeAndWL[identifier]['wl']
    return time, wl



In [6]:
##PLOTTING FUNCTIONS###

In [16]:
def plot_shift(spectrum, wl, n, **kwargs):
    for i in np.arange(0,len(spectrum),n):
        if 'color' in kwargs:
            colors = kwargs.get('color')
        else:
            colors = plt.cm.RdYlBu(round(1 - i/700, 2))
        plt.plot(wl, spectrum.iloc[i,:].T, zorder=len(spectrum)-i, color=colors)
        plt.title(kwargs.get('title'))
        plt.xlabel(kwargs.get('xlabel'))
        plt.ylabel(kwargs.get('ylabel'))

In [2]:
"""
Takes a spectrum from PrepareSpectra (with generated time and wavelengths) and plots it with colors representing change over time
"""
def ShiftOverTime(spectrum, time, wl, string, Title, Title2):
    time = time/60
    
    fig, ax = plt.subplots(figsize = (10,3))
    
    plot_shift(spectrum*1000,wl, 10 )

    wrapped_title = "\n".join(wrap(Title, 40))
    plt.title(wrapped_title, fontsize=14, y = 1.02, loc = 'left')
    fig.suptitle(Title2, fontsize=14, y = 0.85, x = 0.20)
    
    plt.xlabel('Wavenumber (cm\N{SUPERSCRIPT MINUS}\N{SUPERSCRIPT ONE})')
    plt.ylabel('Absorbance (mOD)')

    N = 1000
    cmap = plt.get_cmap('RdYlBu_r',N)
    norm = mpl.colors.Normalize(vmin=min(time),vmax=max(time))
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ticks=np.arange(0,max(time), 10), boundaries=np.arange(-0.05,max(time),.0005))
    cbar.set_label('Time (min)')


In [22]:
def plot_any_peaks_over_time(time, peaks, description, pdf, **kwargs):
    with_plot = kwargs.get("with_plot")
    with_report = kwargs.get("with_report")
    to_save = kwargs.get("to_save")
    params = fit_exponential_model(peaks, time, with_plot, with_report, 1)
    if to_save == True: 
        time_label = '$\\tau_g$ = ' +str(round(params[2]/60,2)) +'min'
    else:
        time_label = None
        

    plt.plot(time/60, peaks, label= time_label)
    plt.xlabel('time (min)', fontsize = 14)
    plt.ylabel('wavenumber (cm\N{SUPERSCRIPT MINUS}\N{SUPERSCRIPT ONE})', fontsize = 14)
    #plt.ylim(2220,2229.5)
    
    if to_save == False:
        plt.legend(loc = 'best')
        wrapped_title = "\n".join(wrap(description, 40))
        plt.title(wrapped_title, fontsize=14, y = 1.02, loc = 'left')
        txt = 'TimeConstant: '+str(round(params[2]/60,2)) +'minutes'
        plt.text(100, 5, txt, ha='center', va='center', transform=None)

        pdf.savefig(bbox_inches = "tight")

In [ ]:
def plotPeaksOverTime(All_Prepped_Spectra, AllTimeAndWL, pdf):
    for key in All_Prepped_Spectra.keys():
        ave = average_of_prepped_spectra(All_Prepped_Spectra, key)
        time, wl = getTimeAndWL(key, AllTimeAndWL)
        spectrum = ave
        time = time/60
        fig, ax = plt.subplots(figsize = (10,3))

        plot_shift(spectrum*1000,wl, 10 )
        ax.margins(x=0)

        ax.tick_params(axis="x", labelsize=14)
        ax.tick_params(axis="y", labelsize=14)
        #ax.title(title, fontsize=20, y = 1.02, loc = 'left')
        fig.suptitle(key, fontsize=14, y = 0.85, x = 0.20)

        plt.xlabel('wavenumber (cm\N{SUPERSCRIPT MINUS}\N{SUPERSCRIPT ONE})', fontsize = 14)
        plt.ylabel('absorbance (10\N{SUPERSCRIPT MINUS}\N{SUPERSCRIPT THREE})', fontsize = 14)

        N = 1000
        cmap = plt.get_cmap('RdYlBu_r',N)
        norm = mpl.colors.Normalize(vmin=min(time),vmax=max(time))
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        cbar = plt.colorbar(sm, ticks=np.arange(0,max(time), 10), boundaries=np.arange(-0.05,max(time),.0005), pad = 0.01);
        cbar.set_label('time (min)', fontsize = 14)
        #plt.savefig(key+'PeakShiftAverage.png', bbox_inches='tight', pad_inches=0.1)
        #plt.show()
        pdf.savefig(bbox_inches='tight', pad_inches=1.0)


def plotPeakShiftAverage(All_Prepped_Spectra, AllTimeAndWL, identifier, title, title2, pdf):
    ave = average_of_prepped_spectra(All_Prepped_Spectra, identifier)
    time, wl = getTimeAndWL(identifier, AllTimeAndWL)
    ShiftOverTime(ave, time, wl, identifier, title, title2)
    #plt.savefig(identifier+'PeakShiftAverage.png', bbox_inches='tight', pad_inches=0.1)
    #plt.show()
    pdf.savefig(bbox_inches='tight', pad_inches=1.0)


def plotUnaveragedInitFin(All_Prepped_Spectra, AllTimeAndWL, identifier, description, pdf):
    time, wl = getTimeAndWL(identifier, AllTimeAndWL)    
    
    for individual_spectrum in All_Prepped_Spectra[identifier].keys():
        if description == 'default':
            title = 'Initial and Final Spectrum From Time Resolved Data Set ' + identifier + ' '+ individual_spectrum
        elif description == 'none':
            title = ''
        else:
            title = description
        spectraDict = All_Prepped_Spectra[identifier]
        plt.figure()
        plt.plot(wl, spectraDict[individual_spectrum].iloc[0, :], label = 'initial spectrum')
        plt.plot(wl, spectraDict[individual_spectrum].iloc[len(spectraDict[individual_spectrum])-1, :], label = 'final spectrum')
        plt.legend(loc = 'best')


        wrapped_title = "\n".join(wrap(title, 40))
        plt.title(wrapped_title, fontsize=14, y=1.02, loc = 'left')
        
        pdf.savefig(bbox_inches='tight', pad_inches=0.1)

def combined_time_plots(All_Prepped_Spectra, tAndWL, paramsDict, TRP, identifier, name, method, pdf):
    time, wl = getTimeAndWL(identifier, tAndWL)
    time_constants = dict()
    if method == 'HalfIntegral':
        peaks, time = PeakFromHalfIntegral(identifier, All_Prepped_Spectra, tAndWL, TRP)
    if method == 'SingleGaussian':
        peaks, time = PeakFromSingleGaussian(identifier, All_Prepped_Spectra, tAndWL, paramsDict)
    plot_any_peaks_over_time(time, peaks, name, pdf, to_save = False,  with_plot = True, with_report = True)
    time_constants[identifier] = fit_exponential_model(peaks, time, 0,0,1)[2]/60
    stats = GetStats(time_constants)
    print('Mean Time Constant: '+ str(np.round(stats[0],2)), u"\u00B1"+str(np.round(stats[1],2))+' minutes')
    return time_constants

